In [16]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from collections import defaultdict, Counter


def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

# STEPS
- load pcap -> tshark extract
- load tshark csv
- filter to a record only
- key srcip
- apply feature functions
 - number of dst
 - number of A queries
 - number of unique A queries
 - diff all: avg, 50%, min, max
 - most popular query 1,2,3,4,5
 - most popular query diff 1,2,3,4,5: avg
 - least popular query
 - FREQUENCY BASED FEATURES (need to finish resampling code #todo)

In [2]:
# load from data/ and extract to processed/
infile='data/split_6hour_00000_20170206235900.pcap'
outfile='processed/split_6hour_00000_20170206235900.csv'

cmd_extract = 'tshark -r '+infile+' -E separator="|" -T fields -e frame.time_epoch \
-e frame.time_relative -e ip.src -e ip.dst -e dns.qry.name -e dns.qry.type -Y "dns.flags.response eq 0" \
> '+outfile

# ONLY DO THIS FOR NEW PCAP FILES
# subprocess.check_output(cmd_extract, shell=True)

In [3]:
# load csv from processed
df = pd.read_csv("processed/split_hour_00000_20170206235900.csv", sep="|", error_bad_lines=False,
                names=['time', 'time_relative', 'srcip', 'dstip', 'dnsqry', 'dnstype'],
                #dtype={'time': pd.np.float, 'time_relative': pd.np.float64, 'dnstype': pd.np.int},
                ).dropna()

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['time'] = df['time'].astype(float)
df['time_relative'] = df['time_relative'].astype(float)

### Stats by dns.qry.type
- convert qry type hex -> int
- A record == 1
- for 2 or more qry type take the first one after splitting

In [5]:
dnstype_count = df.groupby('dnstype')['dnsqry'].count()
dnstype_count.sort_values(inplace=True, ascending=False)

print "Percentage of A Records = ", dnstype_count.iloc[0]/dnstype_count.sum()*100

dnstype_count.head(15)

Percentage of A Records =  77.8158606734


dnstype
0x00000001               1505933
0x0000001c                196059
0x0000000c                153732
0x0000000f                 37187
0x00000021                 33905
0x00000010                  3572
0x00000006                  3331
0x0000002b                   500
0x000000ff                   319
0x000000f9                   185
0x00000001,0x0000001c        147
0x00000002                   110
0x0000b71f                    36
0x0000a21f                    14
0x0000a11f                    11
Name: dnsqry, dtype: int64

In [6]:
# convert dnstype to int (take only first type in case of 2)
df['dnstype'] = df['dnstype'].apply(lambda x: int( x.split(',')[0], 16))

dnstype_count = df.groupby('dnstype')['dnsqry'].count()
dnstype_count.sort_values(inplace=True, ascending=False)

print "New percentage of A Records = ", dnstype_count.iloc[0]/dnstype_count.sum()*100

New percentage of A Records =  77.8234565834


In [34]:
df.head()

,time,time_relative,srcip,dstip,dnsqry,dnstype
0,1.486444e+09,0.000567,10.9.151.75,10.8.0.5,safebrowsing.google.com,1
1,1.486444e+09,0.000744,140.180.223.155,128.112.129.209,51.255.61.103.sbl-xbl.spamhaus.dnsbl,1
2,1.486444e+09,0.000853,140.180.223.155,128.112.129.209,51.255.61.103.sbl-xbl.spamhaus.dnsbl,1
3,1.486444e+09,0.003419,10.9.38.65,10.8.0.5,tpc.googlesyndication.com,1
4,1.486444e+09,0.003675,10.6.63.107,128.112.129.209,98.63.6.10.in-addr.arpa,12


### filter to A records only

In [7]:
df_a = df[df.dnstype == 1]

## group by srcip as KEY and extract features

In [13]:
df_sample = df_a.head(5000).copy()

In [14]:
grouped = df_sample.groupby('srcip')

In [56]:
# grouped.agg({'C' : np.sum, 'D' : lambda x: np.std(x, ddof=1)})

for srcip, group in grouped:
    # key
    print "\n", srcip
    
    # DSTIP
    # number of unique dstip
    print len( set( group['dstip'] ) )
    
    # most common dstip
    print Counter( group['dstip'] ).most_common(1)[0][0]
    
    # DNS QRY
    # count of total domains
    print group['dnsqry'].count()
    
    # count of unique domains
    print len( set( group['dnsqry'] ))

    # TOP 3 most popular domains
    top3 = Counter( group['dnsqry'] ).most_common(3)
    print top3
    
    # TIME DIFF ALL - DESCRIBE
    #td = group['time'].diff()
    #print td.describe()
    

 
10.6.48.12
1
128.112.129.7
5
5
[('fac221w-acad.Princeton.EDU', 1), ('pcocr203w.Princeton.EDU', 1), ('fac-rds-201w.Princeton.EDU', 1)]

10.6.48.13
1
128.112.128.1
1
1
[('clients1.google.com', 1)]

10.6.48.172
1
128.112.128.1
20
19
[('INR-GLDK6V1.princeton.edu', 2), ('WWS-BFDBPN1.princeton.edu', 1), ('HOU-FC5GKK1.princeton.edu', 1)]

10.6.48.20
1
128.112.128.1
1
1
[('endpoint.ingress.rapid7.com', 1)]

10.6.48.22
1
128.112.128.1
1
1
[('endpoint.ingress.rapid7.com', 1)]

10.6.48.48
1
128.112.128.1
1
1
[('PDOM08.pu.win.princeton.edu', 1)]

10.6.48.80
1
128.112.128.1
24
18
[('essesx201v.princeton.edu', 2), ('essesx210v.princeton.edu', 2), ('essesx224v.princeton.edu', 2)]

10.6.49.1
1
128.112.128.1
1
1
[('clients4.google.com', 1)]

10.6.49.115
1
128.112.128.1
1
1
[('TSM8.PRINCETON.EDU', 1)]

10.6.49.141
1
128.112.128.1
2
2
[('PDOM08.pu.win.princeton.edu', 1), ('PDOM05.pu.win.princeton.edu', 1)]

10.6.49.145
1
128.112.128.1
2
2
[('pdom06.pu.win.princeton.edu', 1), ('win03.win.princeton.edu',